# layout

> Display, position, overflow, z-index and other layout utilities for Tailwind CSS

In [ ]:
#| default_exp utilities.layout

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Union, Literal, List, Dict
from dataclasses import dataclass
from cjm_fasthtml_tailwind.core.base import (
    TailwindScale, combine_classes, StandardUtility, UtilityFactory,
    is_numeric_scale, is_fraction, is_custom_property, is_arbitrary_value
)
from cjm_fasthtml_tailwind.builders.scales import (
    ScaledFactory, DirectionalScaledFactory, ScaleConfig, INSET_CONFIG,
    ScaledUtility, NegativeFactory, SimpleFactory
)

## Display Utilities

Tailwind CSS provides utilities for controlling the display box type of an element.

In [ ]:
#| export
DISPLAY_VALUES = { # Display utilities
    "inline": "inline",
    "block": "block", 
    "inline-block": "inline-block",
    "flow-root": "flow-root",
    "flex": "flex",
    "inline-flex": "inline-flex",
    "grid": "grid",
    "inline-grid": "inline-grid",
    "contents": "contents",
    "table": "table",
    "inline-table": "inline-table",
    "table-caption": "table-caption",
    "table-cell": "table-cell",
    "table-column": "table-column",
    "table-column-group": "table-column-group",
    "table-footer-group": "table-footer-group",
    "table-header-group": "table-header-group",
    "table-row-group": "table-row-group",
    "table-row": "table-row",
    "list-item": "list-item",
    "hidden": "hidden",
    "none": "none"
}

display_tw = SimpleFactory(DISPLAY_VALUES) # The display factory

# Special display utilities
sr_only = "sr-only"  # Screen reader only
not_sr_only = "not-sr-only"  # Not screen reader only

In [ ]:
#| export
def test_layout_display_examples():
    """Test display utilities with various values."""
    # Test display utilities with dot notation
    assert display_tw.block == "block"
    assert display_tw.inline == "inline"
    assert display_tw.flex == "flex"
    assert display_tw.grid == "grid"
    assert display_tw.hidden == "hidden"
    assert display_tw.inline_flex == "inline-flex"
    
    # Table display utilities
    assert display_tw.table == "table"
    assert display_tw.table_row == "table-row"
    assert display_tw.table_cell == "table-cell"
    
    # Special utilities
    assert sr_only == "sr-only"
    assert not_sr_only == "not-sr-only"

# Run the tests
test_layout_display_examples()

## Position Utilities

Control how an element is positioned in the document.

In [ ]:
#| export
POSITION_VALUES = { # Position utilities
    "static": "static",
    "fixed": "fixed",
    "absolute": "absolute",
    "relative": "relative",
    "sticky": "sticky"
}

position = SimpleFactory(POSITION_VALUES) # The position factory

In [ ]:
#| export
def test_layout_position_examples():
    """Test position utilities."""
    # Test position utilities with dot notation
    assert position.static == "static"
    assert position.relative == "relative"
    assert position.absolute == "absolute"
    assert position.fixed == "fixed"
    assert position.sticky == "sticky"

# Run the tests
test_layout_position_examples()

## Inset Utilities (Top/Right/Bottom/Left)

Control the placement of positioned elements.

In [ ]:
#| export
# For inset, we need special handling because it uses hyphens in directional variants
class InsetDirectionalFactory:
    """Special factory for inset utilities that use hyphenated directions."""
    
    def __init__(
        self,
        prefix: str,  # The base prefix ('inset')
        config: ScaleConfig  # Configuration defining valid scales and values
    ):
        "Initialize with prefix and scale configuration."
        self.prefix = prefix
        self.config = config
        
        # Create direction-specific factories with hyphens
        self.x = ScaledFactory(f"{prefix}-x", config)  # horizontal
        self.y = ScaledFactory(f"{prefix}-y", config)  # vertical
    
    def __call__(
        self,
        value: Optional[TailwindScale] = None,  # The value to apply to the inset
        negative: bool = False  # Whether to create a negative variant
    ) -> ScaledUtility:  # A new scaled utility instance
        """Create a utility instance for all directions."""
        return ScaledFactory(self.prefix, self.config)(value, negative)
    
    def __getattr__(
        self,
        name: str  # The attribute name to convert to a utility value
    ) -> ScaledUtility:  # A new scaled utility instance
        """Handle attribute access for named values."""
        return ScaledFactory(self.prefix, self.config).__getattr__(name)
    
    @property
    def negative(
        self
    ) -> 'NegativeFactory':  # A factory for creating negative variants
        """Return a negative variant factory."""
        return NegativeFactory(self.prefix, self.config)

# Inset utilities (top, right, bottom, left)
inset = InsetDirectionalFactory("inset", INSET_CONFIG) # The inset factory for positioning

# Individual direction utilities don't need special handling
top = ScaledFactory("top", INSET_CONFIG) # The top factory
right = ScaledFactory("right", INSET_CONFIG) # The right factory
bottom = ScaledFactory("bottom", INSET_CONFIG) # The bottom factory
left = ScaledFactory("left", INSET_CONFIG) # The left factory
start = ScaledFactory("start", INSET_CONFIG) # The start factory (logical)
end = ScaledFactory("end", INSET_CONFIG) # The end factory (logical)

In [ ]:
#| export
def test_layout_inset_examples():
    """Test inset utilities for positioning elements."""
    # Test inset utilities
    assert str(inset(0)) == "inset-0"
    assert str(inset(4)) == "inset-4"
    assert str(inset("1/2")) == "inset-1/2"
    assert str(inset.auto) == "inset-auto"
    assert str(inset.full) == "inset-full"
    assert str(inset.negative(4)) == "-inset-4"
    
    # Test directional inset
    assert str(inset.x(4)) == "inset-x-4"
    assert str(inset.y(8)) == "inset-y-8"
    
    # Test individual directions
    assert str(top(0)) == "top-0"
    assert str(right(4)) == "right-4"
    assert str(bottom.auto) == "bottom-auto"
    assert str(left.negative(2)) == "-left-2"

# Run the tests
test_layout_inset_examples()

## Overflow Utilities

Control how an element handles content that is too large for the container.

In [ ]:
#| export
OVERFLOW_VALUES = ["auto", "hidden", "clip", "visible", "scroll"] # Overflow values

class OverflowFactory:
    """Factory for overflow utilities with directional support."""
    
    def __init__(self):
        "Initialize with overflow values and directional sub-factories."
        # Create base overflow utilities
        self._values = {value: f"overflow-{value}" for value in OVERFLOW_VALUES}
        
        # Create x and y sub-factories
        self.x = type('OverflowX', (), {
            value: f"overflow-x-{value}" for value in OVERFLOW_VALUES
        })()
        self.y = type('OverflowY', (), {
            value: f"overflow-y-{value}" for value in OVERFLOW_VALUES
        })()
    
    def __getattr__(
        self,
        name: str  # The attribute name to look up in overflow values
    ) -> str:  # The corresponding overflow CSS class
        "Get overflow utility by attribute name."
        if name in self._values:
            return self._values[name]
        raise AttributeError(f"'OverflowFactory' object has no attribute '{name}'")

overflow = OverflowFactory() # The overflow factory

In [ ]:
#| export
def test_layout_overflow_examples():
    """Test overflow utilities for content handling."""
    # Test overflow utilities
    assert overflow.auto == "overflow-auto"
    assert overflow.hidden == "overflow-hidden"
    assert overflow.visible == "overflow-visible"
    
    # Directional overflow utilities
    assert overflow.x.auto == "overflow-x-auto"
    assert overflow.y.scroll == "overflow-y-scroll"
    assert overflow.x.visible == "overflow-x-visible"
    assert overflow.y.visible == "overflow-y-visible"
    assert overflow.y.clip == "overflow-y-clip"

# Run the tests
test_layout_overflow_examples()

## Z-Index Utilities

Control the stack order of an element.

In [ ]:
#| export
Z_INDEX_CONFIG = ScaleConfig( # Z-index configuration
    numeric=True,  # Support numeric values 0-50
    decimals=False,
    fractions=False,
    named=None,
    special={
        "auto": "auto"
    },
    negative=True  # Support negative z-index
)

# Create z-index factory
z = ScaledFactory("z", Z_INDEX_CONFIG) # The z-index factory

In [ ]:
#| export
def test_layout_z_index_examples():
    """Test z-index utilities for stack ordering."""
    # Test z-index utilities
    assert str(z(0)) == "z-0"
    assert str(z(10)) == "z-10"
    assert str(z(20)) == "z-20"
    assert str(z(50)) == "z-50"
    assert str(z.auto) == "z-auto"
    assert str(z.negative(10)) == "-z-10"
    assert str(z("999")) == "z-[999]" # (string number without units)
    assert str(z("[999]")) == "z-[999]" # (explicit arbitrary value)

# Run the tests
test_layout_z_index_examples()

## Float Utilities

Control the wrapping of content around an element.

In [ ]:
#| export
# Float utilities
FLOAT_VALUES = {
    "right": "float-right",
    "left": "float-left",
    "start": "float-start",
    "end": "float-end",
    "none": "float-none"
}

# Create float factory with special prefix handling
class FloatFactory(SimpleFactory):
    """Special factory for float utilities that prepends 'float-' to values."""
    def __getattr__(
        self,
        name: str  # The attribute name to look up in float values
    ) -> str:  # The corresponding float CSS class
        "Get float utility by attribute name."
        return super().__getattr__(name)

float_tw = FloatFactory(FLOAT_VALUES)  # Renamed to avoid conflict with Python's float

## Clear Utilities

Control the wrapping behavior of an element after floating elements.

In [ ]:
#| export
# Clear utilities
CLEAR_VALUES = {
    "left": "clear-left",
    "right": "clear-right",
    "both": "clear-both",
    "start": "clear-start",
    "end": "clear-end",
    "none": "clear-none"
}

# Create clear factory
clear = SimpleFactory(CLEAR_VALUES) # The clear factory

In [ ]:
#| export
def test_layout_float_clear_examples():
    """Test float and clear utilities for content wrapping."""
    # Test float utilities
    assert float_tw.right == "float-right"
    assert float_tw.left == "float-left"
    assert float_tw.start == "float-start"
    assert float_tw.end == "float-end"
    assert float_tw.none == "float-none"
    
    # Test clear utilities
    assert clear.left == "clear-left"
    assert clear.right == "clear-right"
    assert clear.both == "clear-both"
    assert clear.start == "clear-start"
    assert clear.end == "clear-end"
    assert clear.none == "clear-none"

# Run the tests
test_layout_float_clear_examples()

## Object Fit Utilities

Control how a replaced element's content should be resized.

In [ ]:
#| export
# Object fit utilities
OBJECT_FIT_VALUES = {
    "contain": "object-contain",
    "cover": "object-cover",
    "fill": "object-fill",
    "none": "object-none",
    "scale-down": "object-scale-down"
}

# Create object fit factory
object_fit = SimpleFactory(OBJECT_FIT_VALUES) # The object fit factory

## Object Position Utilities

Control how a replaced element's content should be positioned within its container.

In [ ]:
#| export
# Object position utilities - combines fixed positions with custom value support
OBJECT_POSITION_VALUES = {
    "top-left": "object-top-left",
    "top": "object-top",
    "top-right": "object-top-right",
    "left": "object-left",
    "center": "object-center",
    "right": "object-right",
    "bottom-left": "object-bottom-left",
    "bottom": "object-bottom",
    "bottom-right": "object-bottom-right"
}

# Enhanced factory that supports both fixed values and custom positions
class ObjectPositionFactory(SimpleFactory):
    """Factory for object position with both fixed and custom values."""
    
    def __call__(
        self,
        value: str  # Custom position value (e.g., '50% 25%' or CSS variable)
    ) -> str:  # The formatted object position CSS class
        """Handle custom position values like '50% 25%'."""
        if is_custom_property(value):
            return f"object-({value})"
        elif is_arbitrary_value(value) or " " in value:
            return f"object-[{value}]"
        return f"object-{value}"

# Create object position factory
object_position = ObjectPositionFactory(OBJECT_POSITION_VALUES) # The object position factory

In [ ]:
#| export
def test_layout_object_examples():
    """Test object fit and position utilities."""
    # Test object fit utilities
    assert object_fit.contain == "object-contain"
    assert object_fit.cover == "object-cover"
    assert object_fit.fill == "object-fill"
    assert object_fit.none == "object-none"
    assert object_fit.scale_down == "object-scale-down"
    
    # Test object position utilities with dot notation
    assert object_position.center == "object-center"
    assert object_position.top == "object-top"
    assert object_position.bottom_right == "object-bottom-right"
    assert object_position("50% 25%") == "object-[50% 25%]"
    assert object_position("--custom-position") == "object-(--custom-position)"

# Run the tests
test_layout_object_examples()

## Visibility Utilities

Control the visibility of an element.

In [ ]:
#| export
# Visibility utilities
VISIBILITY_VALUES = {
    "visible": "visible",
    "invisible": "invisible",
    "collapse": "collapse"
}

# Create visibility factory
visibility = SimpleFactory(VISIBILITY_VALUES) # The visibility factory

## Box Sizing Utilities

Control how the browser should calculate an element's total size.

In [ ]:
#| export
# Box sizing utilities
BOX_SIZING_VALUES = {
    "border": "box-border",
    "content": "box-content"
}

# Create box sizing factory
box = SimpleFactory(BOX_SIZING_VALUES) # The box sizing factory

In [ ]:
#| export
def test_layout_visibility_examples():
    """Test visibility and box sizing utilities."""
    # Test visibility utilities
    assert visibility.visible == "visible"
    assert visibility.invisible == "invisible"
    assert visibility.collapse == "collapse"
    
    # Test box sizing utilities
    assert box.border == "box-border"
    assert box.content == "box-content"

# Run the tests
test_layout_visibility_examples()

## Isolation Utilities

Control whether an element should explicitly create a new stacking context.

In [ ]:
#| export
# Isolation utilities
ISOLATION_VALUES = {
    "isolate": "isolate",
    "auto": "isolation-auto"
}

# Create isolation factory
isolation = SimpleFactory(ISOLATION_VALUES) # The isolation factory

## Aspect Ratio Utilities

Control the aspect ratio of an element.

In [ ]:
#| export
# Aspect ratio utilities - fixed values with custom ratio support
ASPECT_RATIO_VALUES = {
    "auto": "aspect-auto",
    "square": "aspect-square",
    "video": "aspect-video"
}

# Enhanced factory that supports both fixed values and custom ratios
class AspectRatioFactory(SimpleFactory):
    """Factory for aspect ratio with both fixed and custom values."""
    
    def __call__(
        self,
        value: str  # Custom aspect ratio (e.g., '16/9', '4/3', or CSS variable)
    ) -> str:  # The formatted aspect ratio CSS class
        """Handle custom ratio values like '16/9' or '4/3'."""
        # Handle ratio format
        if "/" in value and not is_arbitrary_value(value):
            return f"aspect-{value}"
        # Handle custom properties
        elif is_custom_property(value):
            return f"aspect-({value})"
        # Handle arbitrary values
        elif is_arbitrary_value(value):
            return f"aspect-[{value}]"
        return f"aspect-{value}"

# Create aspect ratio factory
aspect = AspectRatioFactory(ASPECT_RATIO_VALUES) # The aspect ratio factory

In [ ]:
#| export
def test_layout_aspect_columns_examples():
    """Test aspect ratio and columns utilities."""
    # Test aspect ratio utilities with dot notation
    assert aspect.auto == "aspect-auto"
    assert aspect.square == "aspect-square"
    assert aspect.video == "aspect-video"
    assert aspect("16/9") == "aspect-16/9"
    assert aspect("4/3") == "aspect-4/3"
    assert aspect("--custom") == "aspect-(--custom)"

# Run the tests
test_layout_aspect_columns_examples()

## Columns Utilities

Control the number of columns within an element.

In [ ]:
#| export
from cjm_fasthtml_tailwind.core.base import CONTAINER_SCALES

COLUMNS_CONFIG = ScaleConfig( # Columns configuration with container sizes
    numeric=True,  # Support columns-1 through columns-12
    decimals=False,
    fractions=False,
    named=CONTAINER_SCALES,  # Use all container scales (3xs through 7xl)
    special={
        "auto": "auto"
    },
    negative=False
)

# Create columns factory
columns = ScaledFactory("columns", COLUMNS_CONFIG) # The columns factory

In [ ]:
#| export
def test_layout_columns_examples():
    """Test columns utilities."""
    # Test columns utilities
    assert str(columns(1)) == "columns-1"
    assert str(columns(2)) == "columns-2"
    assert str(columns(3)) == "columns-3"
    assert str(columns.auto) == "columns-auto"
    assert str(columns.xs) == "columns-xs"
    assert str(columns.sm) == "columns-sm"
    assert str(columns.lg) == "columns-lg"
    assert str(columns._3xl) == "columns-3xl"
    assert str(columns._4xl) == "columns-4xl"
    assert str(columns._5xl) == "columns-5xl"
    assert str(columns._6xl) == "columns-6xl"
    assert str(columns._7xl) == "columns-7xl"

# Run the tests
test_layout_columns_examples()

## Break Utilities

Control column and page breaks.

In [ ]:
#| export
# Break utilities - organized by type
BREAK_BEFORE_VALUES = {
    "auto": "break-before-auto",
    "avoid": "break-before-avoid",
    "all": "break-before-all",
    "avoid-page": "break-before-avoid-page",
    "page": "break-before-page",
    "left": "break-before-left",
    "right": "break-before-right",
    "column": "break-before-column"
}

BREAK_AFTER_VALUES = {
    "auto": "break-after-auto",
    "avoid": "break-after-avoid",
    "all": "break-after-all",
    "avoid-page": "break-after-avoid-page",
    "page": "break-after-page",
    "left": "break-after-left",
    "right": "break-after-right",
    "column": "break-after-column"
}

BREAK_INSIDE_VALUES = {
    "auto": "break-inside-auto",
    "avoid": "break-inside-avoid",
    "avoid-page": "break-inside-avoid-page",
    "avoid-column": "break-inside-avoid-column"
}

# Create break factories with sub-properties
class BreakFactory:
    """Factory for break utilities with before, after, and inside sub-factories."""
    
    def __init__(self):
        "Initialize with sub-factories for before, after, and inside breaks."
        self.before = SimpleFactory(BREAK_BEFORE_VALUES)
        self.after = SimpleFactory(BREAK_AFTER_VALUES)
        self.inside = SimpleFactory(BREAK_INSIDE_VALUES)

# Create the break factory
break_util = BreakFactory() # The break factory

## Box Decoration Break Utilities

Control how element fragments should be rendered across multiple lines, columns, or pages.

In [ ]:
#| export
# Box decoration break utilities
BOX_DECORATION_VALUES = {
    "clone": "box-decoration-clone",
    "slice": "box-decoration-slice"
}

# Create box decoration factory
box_decoration = SimpleFactory(BOX_DECORATION_VALUES) # The box decoration factory

## Overscroll Behavior Utilities

Control how the browser behaves when reaching the boundary of a scrolling area.

In [ ]:
#| export
# Overscroll behavior values
OVERSCROLL_VALUES = ["auto", "contain", "none"]

class OverscrollFactory:
    """Factory for overscroll behavior utilities with directional support."""
    
    def __init__(self):
        "Initialize with overscroll values and directional sub-factories."
        # Create base overscroll utilities
        self._values = {value: f"overscroll-{value}" for value in OVERSCROLL_VALUES}
        
        # Create x and y sub-factories
        self.x = type('OverscrollX', (), {
            value: f"overscroll-x-{value}" for value in OVERSCROLL_VALUES
        })()
        self.y = type('OverscrollY', (), {
            value: f"overscroll-y-{value}" for value in OVERSCROLL_VALUES
        })()
    
    def __getattr__(
        self,
        name: str  # The attribute name to look up in overscroll values
    ) -> str:  # The corresponding overscroll CSS class
        "Get overscroll utility by attribute name."
        if name in self._values:
            return self._values[name]
        raise AttributeError(f"'OverscrollFactory' object has no attribute '{name}'")

# Create the overscroll factory
overscroll = OverscrollFactory() # The overscroll factory

In [ ]:
#| export
def test_layout_other_utilities_examples():
    """Test isolation, break, box decoration, and overscroll utilities."""
    # Test isolation utilities
    assert isolation.isolate == "isolate"
    assert isolation.auto == "isolation-auto"
    
    # Test break utilities
    assert break_util.before.auto == "break-before-auto"
    assert break_util.before.page == "break-before-page"
    assert break_util.after.column == "break-after-column"
    assert break_util.inside.avoid == "break-inside-avoid"
    
    # Test box decoration break utilities
    assert box_decoration.clone == "box-decoration-clone"
    assert box_decoration.slice == "box-decoration-slice"
    
    # Test overscroll behavior utilities
    assert overscroll.auto == "overscroll-auto"
    assert overscroll.contain == "overscroll-contain"
    assert overscroll.none == "overscroll-none"
    assert overscroll.x.auto == "overscroll-x-auto"
    assert overscroll.y.contain == "overscroll-y-contain"

# Run the tests
test_layout_other_utilities_examples()

## Practical Examples

Let's see how to use these layout utilities in real FastHTML components:

In [ ]:
from fasthtml.common import Div, Img, Header, Nav, Main, Section, Article, Aside

In [ ]:
#| export
def test_layout_practical_examples():
    """Test layout utilities in practical FastHTML component examples."""
    from fasthtml.common import Div, Img, Header, Nav, Main, Section, Article, Aside
    
    # Fixed header with z-index
    header = Header(
        Nav("Navigation"),
        cls=combine_classes(position.fixed, top(0), left(0), right(0), z(50), "bg-white")
    )
    assert header.attrs['class'] == "fixed top-0 left-0 right-0 z-50 bg-white"
    
    # Sticky sidebar with scroll
    sidebar = Aside(
        "Sidebar content",
        cls=combine_classes(position.sticky, top(20), "h-screen", overflow.y.auto)
    )
    assert sidebar.attrs['class'] == "sticky top-20 h-screen overflow-y-auto"
    
    # Modal overlay with z-index
    modal_overlay = Div(
        Div("Modal content", cls=combine_classes(position.relative, z(10))),
        cls=combine_classes(position.fixed, inset(0), z(40), display_tw.flex, "items-center", "justify-center", "bg-black/50")
    )
    assert modal_overlay.attrs['class'] == "fixed inset-0 z-40 flex items-center justify-center bg-black/50"
    assert modal_overlay.children[0].attrs['class'] == "relative z-10"
    
    # Image with aspect ratio and object fit
    image_container = Div(
        Img(src="image.jpg", cls=combine_classes(object_fit.cover, object_position.center, "w-full", "h-full")),
        cls=combine_classes(aspect.video, overflow.hidden, "rounded-lg")
    )
    assert image_container.attrs['class'] == "aspect-video overflow-hidden rounded-lg"
    assert image_container.children[0].attrs['class'] == "object-cover object-center w-full h-full"
    
    # Multi-column layout
    article = Article(
        "Lorem ipsum dolor sit amet...",
        cls=combine_classes(columns(2), "gap-8", "prose")
    )
    assert article.attrs['class'] == "columns-2 gap-8 prose"

# Run the tests
test_layout_practical_examples()

## Helper Functions

Convenient functions for common layout patterns:

In [ ]:
#| export
def center_absolute(
) -> str:  # Combined CSS classes for centering an element
    """Center an absolutely positioned element."""
    return combine_classes(
        position.absolute, 
        top("1/2"), 
        left("1/2"), 
        "-translate-x-1/2",
        "-translate-y-1/2"
    )

In [ ]:
#| export
def stack_context(
    z_value: int = 10  # The z-index value for the stacking context
) -> str:  # Combined CSS classes for creating a stacking context
    """Create a stacking context with z-index."""
    return combine_classes(position.relative, z(z_value))

In [ ]:
#| export
def sticky_top(
    offset: TailwindScale = 0  # Top offset value (e.g., 0, 4, '1rem')
) -> str:  # Combined CSS classes for sticky positioning
    """Make element sticky at top with optional offset."""
    return combine_classes(position.sticky, top(offset))

In [ ]:
#| export
def full_bleed(
) -> str:  # Combined CSS classes for full-bleed layout
    """Make element break out of container constraints."""
    return combine_classes(position.relative, left("1/2"), right("1/2"), "-mx-[50vw]", "w-screen")

In [ ]:
#| export
def test_layout_helper_examples():
    """Test helper functions for common layout patterns."""
    # Test helper functions
    assert center_absolute() == "absolute top-1/2 left-1/2 -translate-x-1/2 -translate-y-1/2"
    assert stack_context(20) == "relative z-20"
    assert sticky_top(4) == "sticky top-4"
    assert full_bleed() == "relative left-1/2 right-1/2 -mx-[50vw] w-screen"

# Run the tests
test_layout_helper_examples()

## Testing

Let's run comprehensive tests to ensure our layout utilities work correctly:

In [ ]:
#| export
def test_layout_all_examples():
    """Run all layout utility example tests."""
    test_layout_display_examples()
    test_layout_position_examples()
    test_layout_inset_examples()
    test_layout_overflow_examples()
    test_layout_z_index_examples()
    test_layout_float_clear_examples()
    test_layout_object_examples()
    test_layout_visibility_examples()
    test_layout_aspect_columns_examples()
    test_layout_columns_examples()
    test_layout_other_utilities_examples()
    test_layout_helper_examples()
    test_layout_practical_examples()

# Run all tests
test_layout_all_examples()

In [ ]:
print("✅ All layout utility tests passed!")

✅ All layout utility tests passed!


## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()